In [8]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os.path import dirname, join
protoFile = join("pose_deploy_linevec_faster_4_stages.prototxt")
weightsFile = join("pose_iter_160000.caffemodel")
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)


In [9]:
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

In [18]:
def posedetector(frame):
    inWidth = 368
    inHeight = 368
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()
    
    H = output.shape[2]
    W = output.shape[3]
    points = []
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    for i in range(output.shape[1]):
        probMap = output[0, i, :, :]
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H
        if prob > 0.3 :
            cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            points.append((int(x), int(y)))
        else :
            points.append(None)
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        idFrom = BODY_PARTS[partA]
        idTo = BODY_PARTS[partB]
        if points[idFrom] and points[idTo]:
            cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 5)
    cv2.imshow('frame',frame)

In [24]:
cap = cv2.VideoCapture('tennis_video.mp4') 
while(True): 
    ret,frame = cap.read() 
    posedetector(frame) 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cap.release() 
cv2.destroyAllWindows()